# Zwalm

In [ ]:
import xarray as xr
from pathlib import Path
import hvplot.xarray
import numpy as np
import numpy.ma as ma
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

## Make 1 Dataset

In [ ]:
zwalm_xr = xr.open_mfdataset('data/g0_020m_Zwalm/*_110_Zwalm.nc')

In [ ]:
zwalm_xr

## Plotting

In [ ]:
zwalm_xr['g0vv'].hvplot.image(geo = True, tiles = 'OSM', cmap = 'cividis', frame_width = 350, alpha = 0.8)

In [ ]:
zwalm_xr['lia'].hvplot.image(geo = True, tiles = 'OSM', cmap = 'cividis', frame_width = 350)

## Average backscatter timeseries

In [ ]:
zwalm_xr['g0vv_abs'] = 10**(zwalm_xr['g0vv']/10)
zwalm_xr['g0vv_abs'].attrs['grid_mapping'] = 'crs'
zwalm_xr

In [ ]:
zwalm_xr['g0vv_abs'].hvplot.image(geo = True, tiles = 'OSM', cmap = 'cividis', frame_width = 350)

Duidelijk dat veel reflectie in zottegem => overwegen om deze waarden er uit te laten? Want hier is infrastructuur eerder...


In [ ]:
zwalm_xr['g0vv_av_timeseries'] = 10*np.log10(zwalm_xr['g0vv_abs'].mean(dim = ['lat','lon']))
zwalm_xr

In [ ]:
zwalm_xr['g0vv_av_timeseries'].plot()

In [ ]:
zwalm_xr['g0vv_av_timeseries'].hvplot()

## Correlation between angle of incidene and g0vv backscatter

Over the entire dataset

In [ ]:
ma.corrcoef(ma.masked_invalid(zwalm_xr['g0vv'].values.flatten()), 
            ma.masked_invalid(zwalm_xr['lia'].values.flatten()))

So very low correlation of only 0.029! Seems to indicate negligible correlation, which is wanted.

In [ ]:
stacked = zwalm_xr.stack(latlon = ['lat','lon'])
stacked

In [ ]:
corr = xr.corr(stacked['g0vv'],stacked['lia'], dim = "time").unstack()
corr

In [ ]:
zwalm_xr['corr_g0vv_lia'] = corr
zwalm_xr['corr_g0vv_lia'].attrs['grid_mapping'] = 'crs'
zwalm_xr

In [ ]:
fig, ax = plt.subplots()
bins = corr.plot.hist(bins = 'auto', density = True, ax = ax)
ax.set_title('Distribution of correlation between backscatter and angle of incidence for every pixel')

Conclusion: as the correlations seem to be normally distributed around zero, no corrections seems necessary for this correlation? 

In [ ]:
corr.hvplot(geo = True, cmap = 'RdBu', frame_width = 350)